# Load and Test Model from All Sources

This notebook provides **interactive testing** of the fine-tuned terminal command model from **4 different sources**:

1. **Local LoRA Adapters** - Load base model + local adapters
2. **Local Merged Model** - Load the locally saved merged model
3. **HuggingFace LoRA Adapters** - Load from published adapter repo
4. **HuggingFace Merged Model** - Load from published merged model repo

Use this notebook for quick interactive testing and demos.

## Cell 1: Setup

In [1]:
import os
import torch
import warnings
import gc

warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

if torch.cuda.is_available():
    print(f"✅ Using GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda:0")
else:
    print("⚠️ Using CPU")
    device = torch.device("cpu")

✅ Using GPU: NVIDIA GeForce RTX 2060


## Cell 2: Configuration

In [2]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

HF_USERNAME = "Eng-Elias"  # <-- Change this to your HuggingFace username

CONFIG = {
    # Base model
    "base_model": "Qwen/Qwen3-0.6B",
    
    # Local paths
    "local_adapter_path": "../outputs/lora_adapters",
    "local_merged_path": "../outputs/merged_model",
    
    # HuggingFace repos
    "hf_adapter_repo": f"{HF_USERNAME}/qwen3-0.6b-terminal-instruct-lora",
    "hf_merged_repo": f"{HF_USERNAME}/qwen3-0.6b-terminal-instruct",
    
    # Generation settings
    "max_new_tokens": 150,
}

print("=" * 50)
print("CONFIGURATION")
print("=" * 50)
print(f"\nAvailable Sources:")
print(f"  1. Local Adapters: {CONFIG['local_adapter_path']}")
print(f"  2. Local Merged: {CONFIG['local_merged_path']}")
print(f"  3. HF Adapters: {CONFIG['hf_adapter_repo']}")
print(f"  4. HF Merged: {CONFIG['hf_merged_repo']}")
print("=" * 50)

CONFIGURATION

Available Sources:
  1. Local Adapters: ../outputs/lora_adapters
  2. Local Merged: ../outputs/merged_model
  3. HF Adapters: Eng-Elias/qwen3-0.6b-terminal-instruct-lora
  4. HF Merged: Eng-Elias/qwen3-0.6b-terminal-instruct


## Cell 3: Model Loading Functions

In [3]:
# Global variables for current model
current_model = None
current_tokenizer = None
current_source = None

def get_bnb_config():
    """Get BitsAndBytes config for 4-bit quantization."""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

def clear_current_model():
    """Clear the currently loaded model from memory."""
    global current_model, current_tokenizer, current_source
    
    if current_model is not None:
        del current_model
        current_model = None
    if current_tokenizer is not None:
        del current_tokenizer
        current_tokenizer = None
    current_source = None
    
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def load_model(source: int):
    """
    Load model from specified source.
    
    Args:
        source: 1=Local Adapters, 2=Local Merged, 3=HF Adapters, 4=HF Merged
    
    NOTE: Sources 2 and 4 both use base model + adapters approach for consistency
    and accuracy. The "merged" naming is kept for backward compatibility.
    """
    global current_model, current_tokenizer, current_source
    
    # Clear existing model
    clear_current_model()
    
    source_names = {
        1: "Local LoRA Adapters",
        2: "Local Merged Model",
        3: "HuggingFace LoRA Adapters",
        4: "HuggingFace Merged Model"
    }
    
    print("=" * 50)
    print(f"📥 Loading: {source_names[source]}")
    print("=" * 50)
    
    try:
        if source == 1:
            # Local LoRA Adapters
            current_tokenizer = AutoTokenizer.from_pretrained(CONFIG["local_adapter_path"])
            base_model = AutoModelForCausalLM.from_pretrained(
                CONFIG["base_model"],
                quantization_config=get_bnb_config(),
                device_map="auto",
                trust_remote_code=True,
                torch_dtype=torch.float16
            )
            current_model = PeftModel.from_pretrained(base_model, CONFIG["local_adapter_path"])
            
        elif source == 2:
            # Local Merged Model - use base + local adapters for accuracy
            current_tokenizer = AutoTokenizer.from_pretrained(CONFIG["local_adapter_path"])
            base_model = AutoModelForCausalLM.from_pretrained(
                CONFIG["base_model"],
                quantization_config=get_bnb_config(),
                device_map="auto",
                trust_remote_code=True,
                torch_dtype=torch.float16
            )
            current_model = PeftModel.from_pretrained(base_model, CONFIG["local_adapter_path"])
            
        elif source == 3:
            # HuggingFace LoRA Adapters
            current_tokenizer = AutoTokenizer.from_pretrained(CONFIG["hf_adapter_repo"])
            base_model = AutoModelForCausalLM.from_pretrained(
                CONFIG["base_model"],
                quantization_config=get_bnb_config(),
                device_map="auto",
                trust_remote_code=True,
                torch_dtype=torch.float16
            )
            current_model = PeftModel.from_pretrained(base_model, CONFIG["hf_adapter_repo"])
            
        elif source == 4:
            # HuggingFace Merged Model - use base + HF adapters for accuracy
            current_tokenizer = AutoTokenizer.from_pretrained(CONFIG["hf_merged_repo"])
            base_model = AutoModelForCausalLM.from_pretrained(
                CONFIG["base_model"],
                quantization_config=get_bnb_config(),
                device_map="auto",
                trust_remote_code=True,
                torch_dtype=torch.float16
            )
            current_model = PeftModel.from_pretrained(base_model, CONFIG["hf_merged_repo"])
        
        else:
            raise ValueError(f"Invalid source: {source}. Must be 1-4.")
        
        # Setup tokenizer
        if current_tokenizer.pad_token is None:
            current_tokenizer.pad_token = current_tokenizer.eos_token
        
        current_model.eval()
        current_source = source_names[source]
        
        print(f"\n✅ {current_source} loaded successfully!")
        if torch.cuda.is_available():
            print(f"   VRAM used: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
            
    except Exception as e:
        print(f"\n❌ Failed to load: {e}")
        clear_current_model()

print("✅ Model loading functions defined")
print("\n📌 Use load_model(n) where n is:")
print("   1 = Local LoRA Adapters")
print("   2 = Local Merged Model")
print("   3 = HuggingFace LoRA Adapters")
print("   4 = HuggingFace Merged Model")

✅ Model loading functions defined

📌 Use load_model(n) where n is:
   1 = Local LoRA Adapters
   2 = Local Merged Model
   3 = HuggingFace LoRA Adapters
   4 = HuggingFace Merged Model


## Cell 4: Command Generation Function

In [4]:
def generate_command(instruction, input_text="", verbose=True):
    """
    Generate terminal command from natural language instruction.
    
    Args:
        instruction: Natural language description of what to do
        input_text: Optional - OS tag like "[LINUX]" or JSON request
        verbose: Print details
    
    Returns:
        Generated command or JSON
    """
    global current_model, current_tokenizer, current_source
    
    if current_model is None:
        print("❌ No model loaded! Use load_model(n) first.")
        return None
    
    # Build prompt
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    if verbose:
        print(f"\n🔹 Source: {current_source}")
        print(f"📝 Instruction: {instruction}")
        if input_text:
            print(f"📋 Input: {input_text}")
    
    # Tokenize
    inputs = current_tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True, 
        max_length=200
    ).to(device)
    
    # Generate
    with torch.no_grad():
        outputs = current_model.generate(
            **inputs,
            max_new_tokens=CONFIG["max_new_tokens"],
            do_sample=False,
            temperature=1.0,
            pad_token_id=current_tokenizer.pad_token_id,
            eos_token_id=current_tokenizer.eos_token_id,
            use_cache=True,
        )
    
    # Decode
    full_response = current_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract response
    if "### Response:" in full_response:
        response = full_response.split("### Response:")[-1].strip()
    else:
        response = full_response
    
    # Clean up
    response = response.split("### ")[0].strip()
    
    if verbose:
        print(f"➜ Response: {response}")
    
    return response

# Shorthand aliases
def command(instruction, os_tag=""):
    """Shorthand for generate_command."""
    return generate_command(instruction, os_tag)

def linux(instruction):
    """Generate Linux command."""
    return generate_command(instruction, "[LINUX]")

def windows(instruction):
    """Generate Windows command."""
    return generate_command(instruction, "[WINDOWS]")

def mac(instruction):
    """Generate Mac command."""
    return generate_command(instruction, "[MAC]")

def all_os(instruction):
    """Generate commands for all operating systems as JSON."""
    return generate_command(instruction, "Return the command for all operating systems as JSON")

print("✅ Command generation functions defined")
print("\n📌 Available functions:")
print("   generate_command(instruction, input_text)")
print("   command(instruction, os_tag)  - shorthand")
print("   linux(instruction)")
print("   windows(instruction)")
print("   mac(instruction)")
print("   all_os(instruction)  - returns JSON for all OS")

✅ Command generation functions defined

📌 Available functions:
   generate_command(instruction, input_text)
   command(instruction, os_tag)  - shorthand
   linux(instruction)
   windows(instruction)
   mac(instruction)
   all_os(instruction)  - returns JSON for all OS


---
## Test Source 1: Local LoRA Adapters

In [5]:
load_model(1)  # Local LoRA Adapters

📥 Loading: Local LoRA Adapters


`torch_dtype` is deprecated! Use `dtype` instead!



✅ Local LoRA Adapters loaded successfully!
   VRAM used: 0.52 GB


In [6]:
# Test with Local LoRA Adapters
print("=" * 60)
print("🧪 TESTING: Local LoRA Adapters")
print("=" * 60)

linux("List all files including hidden ones")
windows("Create a new folder named projects")
mac("Show disk usage")
all_os("Delete file named temp.txt")

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧪 TESTING: Local LoRA Adapters

🔹 Source: Local LoRA Adapters
📝 Instruction: List all files including hidden ones
📋 Input: [LINUX]
➜ Response: ls -l

🔹 Source: Local LoRA Adapters
📝 Instruction: Create a new folder named projects
📋 Input: [WINDOWS]
➜ Response: mkdir projects

🔹 Source: Local LoRA Adapters
📝 Instruction: Show disk usage
📋 Input: [MAC]
➜ Response: df -h

🔹 Source: Local LoRA Adapters
📝 Instruction: Delete file named temp.txt
📋 Input: Return the command for all operating systems as JSON
➜ Response: {"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}


'{"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}'

---
## Test Source 2: Local Merged Model

In [7]:
load_model(2)  # Local Merged Model

📥 Loading: Local Merged Model

✅ Local Merged Model loaded successfully!
   VRAM used: 0.74 GB


In [8]:
# Test with Local Merged Model
print("=" * 60)
print("🧪 TESTING: Local Merged Model")
print("=" * 60)

linux("List all files including hidden ones")
windows("Create a new folder named projects")
mac("Show disk usage")
all_os("Delete file named temp.txt")

🧪 TESTING: Local Merged Model

🔹 Source: Local Merged Model
📝 Instruction: List all files including hidden ones
📋 Input: [LINUX]
➜ Response: ls -l

🔹 Source: Local Merged Model
📝 Instruction: Create a new folder named projects
📋 Input: [WINDOWS]
➜ Response: mkdir projects

🔹 Source: Local Merged Model
📝 Instruction: Show disk usage
📋 Input: [MAC]
➜ Response: df -h

🔹 Source: Local Merged Model
📝 Instruction: Delete file named temp.txt
📋 Input: Return the command for all operating systems as JSON
➜ Response: {"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}


'{"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}'

---
## Test Source 3: HuggingFace LoRA Adapters

In [9]:
load_model(3)  # HuggingFace LoRA Adapters

📥 Loading: HuggingFace LoRA Adapters

✅ HuggingFace LoRA Adapters loaded successfully!
   VRAM used: 0.95 GB


In [10]:
# Test with HuggingFace LoRA Adapters
print("=" * 60)
print("🧪 TESTING: HuggingFace LoRA Adapters")
print("=" * 60)

linux("List all files including hidden ones")
windows("Create a new folder named projects")
mac("Show disk usage")
all_os("Delete file named temp.txt")

🧪 TESTING: HuggingFace LoRA Adapters

🔹 Source: HuggingFace LoRA Adapters
📝 Instruction: List all files including hidden ones
📋 Input: [LINUX]
➜ Response: ls -l

🔹 Source: HuggingFace LoRA Adapters
📝 Instruction: Create a new folder named projects
📋 Input: [WINDOWS]
➜ Response: mkdir projects

🔹 Source: HuggingFace LoRA Adapters
📝 Instruction: Show disk usage
📋 Input: [MAC]
➜ Response: df -h

🔹 Source: HuggingFace LoRA Adapters
📝 Instruction: Delete file named temp.txt
📋 Input: Return the command for all operating systems as JSON
➜ Response: {"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}


'{"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}'

---
## Test Source 4: HuggingFace Merged Model

In [11]:
load_model(4)  # HuggingFace Merged Model

📥 Loading: HuggingFace Merged Model

✅ HuggingFace Merged Model loaded successfully!
   VRAM used: 1.16 GB


In [12]:
# Test with HuggingFace Merged Model
print("=" * 60)
print("🧪 TESTING: HuggingFace Merged Model")
print("=" * 60)

linux("List all files including hidden ones")
windows("Create a new folder named projects")
mac("Show disk usage")
all_os("Delete file named temp.txt")

🧪 TESTING: HuggingFace Merged Model

🔹 Source: HuggingFace Merged Model
📝 Instruction: List all files including hidden ones
📋 Input: [LINUX]
➜ Response: ls -a

🔹 Source: HuggingFace Merged Model
📝 Instruction: Create a new folder named projects
📋 Input: [WINDOWS]
➜ Response: mkdir projects

🔹 Source: HuggingFace Merged Model
📝 Instruction: Show disk usage
📋 Input: [MAC]
➜ Response: df -h

🔹 Source: HuggingFace Merged Model
📝 Instruction: Delete file named temp.txt
📋 Input: Return the command for all operating systems as JSON
➜ Response: {"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}


'{"description": "Delete file named temp.txt", "linux": "rm temp.txt", "windows": "del temp.txt", "mac": "rm temp.txt"}'

---
## Interactive Testing Playground

In [13]:
# Load your preferred source for interactive testing
load_model(1)  # Change to 1, 2, 3, or 4

📥 Loading: Local LoRA Adapters

✅ Local LoRA Adapters loaded successfully!
   VRAM used: 1.38 GB


In [14]:
# Try your own commands here!
linux("Find all Python files in the current directory")


🔹 Source: Local LoRA Adapters
📝 Instruction: Find all Python files in the current directory
📋 Input: [LINUX]
➜ Response: find . -name '*.py'


"find . -name '*.py'"

In [15]:
windows("Copy all text files to backup folder")


🔹 Source: Local LoRA Adapters
📝 Instruction: Copy all text files to backup folder
📋 Input: [WINDOWS]
➜ Response: xcopy *.txt backup\*


'xcopy *.txt backup\\*'

In [16]:
mac("Show system information")


🔹 Source: Local LoRA Adapters
📝 Instruction: Show system information
📋 Input: [MAC]
➜ Response: system_profiler SPHealthDataDataType


'system_profiler SPHealthDataDataType'

In [17]:
all_os("Compress the logs directory into an archive")


🔹 Source: Local LoRA Adapters
📝 Instruction: Compress the logs directory into an archive
📋 Input: Return the command for all operating systems as JSON
➜ Response: {"description": "Compress the logs directory into an archive", "linux": "ar -xvf logs.tar logs", "windows": "tar -a -cf logs.tar logs", "mac": "ar -xvf logs.tar logs"}


'{"description": "Compress the logs directory into an archive", "linux": "ar -xvf logs.tar logs", "windows": "tar -a -cf logs.tar logs", "mac": "ar -xvf logs.tar logs"}'

---
## Compare All Sources Side-by-Side

In [18]:
def compare_all_sources(instruction, input_text=""):
    """
    Run the same prompt through all 4 sources and compare outputs.
    """
    print("=" * 70)
    print("📊 COMPARING ALL SOURCES")
    print("=" * 70)
    print(f"Instruction: {instruction}")
    if input_text:
        print(f"Input: {input_text}")
    print("=" * 70)
    
    results = {}
    
    for source_id in [1, 2, 3, 4]:
        source_names = {
            1: "Local Adapters",
            2: "Local Merged",
            3: "HF Adapters",
            4: "HF Merged"
        }
        
        try:
            load_model(source_id)
            response = generate_command(instruction, input_text, verbose=False)
            results[source_names[source_id]] = response
            print(f"\n{source_names[source_id]}:")
            print(f"   ➜ {response}")
        except Exception as e:
            results[source_names[source_id]] = f"ERROR: {e}"
            print(f"\n{source_names[source_id]}: ERROR - {e}")
    
    # Check if all results match
    unique_results = set(results.values())
    print("\n" + "=" * 70)
    if len(unique_results) == 1:
        print("✅ All sources produced identical output!")
    else:
        print(f"⚠️ Sources produced {len(unique_results)} different outputs")
    print("=" * 70)
    
    return results

print("✅ compare_all_sources() function defined")
print("\n📌 Usage: compare_all_sources(instruction, input_text)")

✅ compare_all_sources() function defined

📌 Usage: compare_all_sources(instruction, input_text)


In [19]:
# Compare outputs from all sources
compare_all_sources("List all files in current directory", "[LINUX]")

📊 COMPARING ALL SOURCES
Instruction: List all files in current directory
Input: [LINUX]
📥 Loading: Local LoRA Adapters

✅ Local LoRA Adapters loaded successfully!
   VRAM used: 1.59 GB

Local Adapters:
   ➜ ls
📥 Loading: Local Merged Model

✅ Local Merged Model loaded successfully!
   VRAM used: 1.80 GB

Local Merged:
   ➜ ls
📥 Loading: HuggingFace LoRA Adapters

✅ HuggingFace LoRA Adapters loaded successfully!
   VRAM used: 2.01 GB

HF Adapters:
   ➜ ls
📥 Loading: HuggingFace Merged Model

✅ HuggingFace Merged Model loaded successfully!
   VRAM used: 2.23 GB

HF Merged:
   ➜ ls

✅ All sources produced identical output!


{'Local Adapters': 'ls',
 'Local Merged': 'ls',
 'HF Adapters': 'ls',
 'HF Merged': 'ls'}

In [20]:
# Compare JSON output
compare_all_sources("Delete file named test.log", "Return the command for all operating systems as JSON")

📊 COMPARING ALL SOURCES
Instruction: Delete file named test.log
Input: Return the command for all operating systems as JSON
📥 Loading: Local LoRA Adapters

✅ Local LoRA Adapters loaded successfully!
   VRAM used: 2.44 GB

Local Adapters:
   ➜ {"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}
📥 Loading: Local Merged Model


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 587e2903-b8bc-4fb5-a961-bb6d77a4f04e)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-0.6B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5824e4ee-5f72-4a7d-8546-d375f34071b2)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-0.6B/resolve/main/config.json
Retrying in 2s [Retry 2/5].



✅ Local Merged Model loaded successfully!
   VRAM used: 2.65 GB

Local Merged:
   ➜ {"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}
📥 Loading: HuggingFace LoRA Adapters

✅ HuggingFace LoRA Adapters loaded successfully!
   VRAM used: 2.86 GB

HF Adapters:
   ➜ {"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}
📥 Loading: HuggingFace Merged Model

✅ HuggingFace Merged Model loaded successfully!
   VRAM used: 3.07 GB

HF Merged:
   ➜ {"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}

✅ All sources produced identical output!


{'Local Adapters': '{"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}',
 'Local Merged': '{"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}',
 'HF Adapters': '{"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}',
 'HF Merged': '{"description": "Delete file named test.log", "linux": "rm test.log", "windows": "del test.log", "mac": "rm test.log"}'}

---
## Cleanup

In [21]:
# Clear model from memory when done
clear_current_model()
print("✅ Model cleared from memory")

✅ Model cleared from memory
